In [41]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
import time 
import pandas as pd
import re 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options


chrome_options = Options()
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-gpu")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)



In [42]:
chrome_driver_path = "/Users/krishaantpathmanathan/ECON323 final projext/chromedriver"

service = Service(chrome_driver_path)

driver = webdriver.Chrome(service=service)

# creating empty dataframes
all_reviews = pd.DataFrame(columns=["Professor_ID", 
                                     "Professor_Name", 
                                     "University", 
                                     "Department", 
                                     "Quality", 
                                     "Difficulty", 
                                     "Emotion", 
                                     "Class_Name", 
                                     "Comment", 
                                     "Thumbs_Up", 
                                     "Thumbs_Down", 
                                     "Review_Date"])

# UBC is 1413
driver.get("https://www.ratemyprofessors.com/search/professors/1413?q=*")

time.sleep(5)

In [43]:

# closing pop-ups 
try:
    close_button = driver.find_element(By.XPATH, "//div[starts-with(@class, 'FullPageModal__')]//button")
    close_button.click()
except:
    pass


In [33]:
# Load previously saved URLs if they exist
try:
    with open("teacher_urls.txt", "r") as f:
        all_urls = f.read().splitlines()
    print(f"✅ Loaded {len(all_urls)} previously saved URLs.")
except FileNotFoundError:
    all_urls = []
    print("📂 No previous URLs found. Starting fresh.")

# Clicking show more 
for i in range(100000000):  
    try:
        # Find the 'Show More' button and click it
        show_more = driver.find_element(By.XPATH, "//button[text()='Show More']")
        driver.execute_script("arguments[0].scrollIntoView(true);", show_more)
        time.sleep(1)
        show_more.click()
        time.sleep(1.5)

        # Collect all teacher URLs from the current page
        teacher_cards = driver.find_elements(By.XPATH, "//a[starts-with(@class, 'TeacherCard__StyledTeacherCard')]")
        new_urls = [card.get_attribute("href") for card in teacher_cards]

        # Filter new URLs to exclude duplicates
        unique_new_urls = [url for url in new_urls if url not in all_urls]

        # If new URLs are found, add them to the list and save to file
        if unique_new_urls:
            all_urls.extend(unique_new_urls)  

            with open("teacher_urls.txt", "w") as f:
                for url in all_urls:
                    f.write(url + "\n")
            
    except:
        break  

# Print all collected URLs
print(all_urls)


📂 No previous URLs found. Starting fresh.
['https://www.ratemyprofessors.com/professor/1328688', 'https://www.ratemyprofessors.com/professor/1548820', 'https://www.ratemyprofessors.com/professor/55311', 'https://www.ratemyprofessors.com/professor/84651', 'https://www.ratemyprofessors.com/professor/1844857', 'https://www.ratemyprofessors.com/professor/138233', 'https://www.ratemyprofessors.com/professor/2424521', 'https://www.ratemyprofessors.com/professor/35546', 'https://www.ratemyprofessors.com/professor/383338', 'https://www.ratemyprofessors.com/professor/2777762', 'https://www.ratemyprofessors.com/professor/2340525', 'https://www.ratemyprofessors.com/professor/2323949', 'https://www.ratemyprofessors.com/professor/174244', 'https://www.ratemyprofessors.com/professor/1810761', 'https://www.ratemyprofessors.com/professor/2722392', 'https://www.ratemyprofessors.com/professor/2627665', 'https://www.ratemyprofessors.com/professor/2377536', 'https://www.ratemyprofessors.com/professor/1968

In [40]:

# Load previously saved URLs
with open("teacher_urls.txt", "r") as f:
    teacher_urls = f.read().splitlines()

# Loop teacher_urls

for url in teacher_urls:
    
    # oppening induvidual urls 
    try:
        print(f"opening {url}")
        driver.get(url)
        time.sleep(2)
    except Exception as e: # if it fails here it goes back uo and try's the next prof url 
        print(f"failed to load {url}. {e}")
        continue  

    # professor details (top card)
    professor_id = re.findall(r'\d+$', url)[0]
    try:
        professor_name = driver.find_element(By.XPATH, "//div[starts-with(@class, 'NameTitle__Name')]").text #those htmls that you looked for 
        department = driver.find_element(By.XPATH, "//div[starts-with(@class, 'NameTitle__Title')]//span//b").text
        university = driver.find_element(By.XPATH, "//div[starts-with(@class, 'NameTitle__Title')]//a").text
    except NoSuchElementException:
        print("failed - prof. details")
        continue  

    # load more reviews (doesn't work at the moment)
    while True:
        try:
            load_more = driver.find_element(By.XPATH, "//button[text()='Load More Ratings']")
            driver.execute_script("arguments[0].scrollIntoView(true);", load_more)
            load_more.click()
            print("loading more")
            time.sleep(random.uniform(2, 4))
        except:
            print("no more to load")
            break

    # reviews (induvidual reviews)
    reviews = driver.find_elements(By.XPATH, "//div[starts-with(@class, 'Rating__RatingBody')]")
    print(f"there are {len(reviews)} reviews for {professor_name}")

    # Process each review (for each induvidual review)
    for review in reviews:
        # Initialize variables with None
        quality = difficulty = class_name = comment = thumbs_up = thumbs_down = review_date = None

        try:
            quality = review.find_element(By.XPATH, "(.//div[starts-with(@class, 'CardNumRating')])[3]").text
        except NoSuchElementException:
            pass

        try: 
            difficulty = review.find_element(By.XPATH, "(.//div[starts-with(@class, 'CardNumRating')])[6]").text
        except NoSuchElementException:
            pass

        try:    
            class_name = review.find_element(By.XPATH, "(.//div[starts-with(@class,'RatingHeader__StyledClass')])[2]").text
        except NoSuchElementException:
            pass

        try:     
            comment = review.find_element(By.XPATH, ".//div[starts-with(@class, 'Comments__StyledComments')]").text
        except NoSuchElementException:
            pass

        try:    
            thumbs_up = review.find_element(By.XPATH, "(.//div[starts-with(@class, 'Thumbs__HelpTotal')])[1]").text
        except NoSuchElementException:
            thumbs_up = "0"

        try:    
            thumbs_down = review.find_element(By.XPATH, "(.//div[starts-with(@class, 'Thumbs__HelpTotal')])[2]").text
        except NoSuchElementException:
            thumbs_down = "0"

        try:    
            review_date = review.find_element(By.XPATH, "(.//div[starts-with(@class, 'TimeStamp')])[2]").text
        except NoSuchElementException:
            pass

        # Add review data to DataFrame
        all_reviews = pd.concat([
            all_reviews,
            pd.DataFrame([{
                "Professor_ID": professor_id,
                "Professor_Name": professor_name,
                "University": university,
                "Department": department,
                "Quality": float(quality) if quality and quality.replace('.', '', 1).isdigit() else None,
                "Difficulty": float(difficulty) if difficulty and difficulty.replace('.', '', 1).isdigit() else None,
                "Class_Name": class_name,
                "Comment": comment,
                "Thumbs_Up": int(thumbs_up) if thumbs_up.isdigit() else None,
                "Thumbs_Down": int(thumbs_down) if thumbs_down.isdigit() else None,
                "Review_Date": review_date
            }])
        ], ignore_index=True)
    
    print('done9')

# Save the data to a CSV file
all_reviews.to_csv("RMP_reviews.csv", index=False)
print("ALLL DONE BABY!")

# Close the driver
driver.quit()

opening https://www.ratemyprofessors.com/professor/1328688
no more to load
there are 20 reviews for Fok-Shuen Leung


/var/folders/pk/gm23sx4x0k33f49dv5hh292m0000gn/T/ipykernel_33371/1211505002.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_reviews = pd.concat([


done9
opening https://www.ratemyprofessors.com/professor/1548820
failed to load https://www.ratemyprofessors.com/professor/1548820. HTTPConnectionPool(host='localhost', port=53407): Read timed out. (read timeout=120)
opening https://www.ratemyprofessors.com/professor/55311
no more to load
there are 20 reviews for Michael Hasinoff
done9
opening https://www.ratemyprofessors.com/professor/84651
no more to load
there are 20 reviews for Tim Inglis
done9
opening https://www.ratemyprofessors.com/professor/1844857
no more to load
there are 20 reviews for Marina Adshade
done9
opening https://www.ratemyprofessors.com/professor/138233
no more to load
there are 20 reviews for Barbara Cox
done9
opening https://www.ratemyprofessors.com/professor/2424521
no more to load
there are 20 reviews for Anne Thomas
done9
opening https://www.ratemyprofessors.com/professor/35546
no more to load
there are 20 reviews for Mike Feeley
done9
opening https://www.ratemyprofessors.com/professor/383338
no more to load
t

NameError: name 'NoSuchElementException' is not defined

In [11]:
# Save the data to a CSV file
all_reviews.to_csv("Test_Reviews.csv", index=False)

# Close the driver
driver.quit()

print("END !")


END !
